##**General information**

This Colab notebook enables to design sequence of IDPs with target scaling exponents ($\nu$) and radius of gyration ($r_g$).

The user needs to provide as input a starting sequence and a target value for $\nu$. The starting sequence can be evolved with swap moves or single point mutations.



In [19]:
#@title <b>Preliminary operations (i)</b>
import random
import subprocess
import os
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import itertools
from joblib import dump, load
!pip install wget
!pip install localcider
!pip install scikit-learn

!pip install idptools-parrot --quiet;
!pip install git+https://git@github.com/idptools/sparrow.git --quiet;
import wget
from localcider.sequenceParameters import SequenceParameters
import matplotlib.pyplot as plt
from google.colab import files
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')
import sklearn



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [20]:
! pip install fair-esm

In [21]:


# Install from GitHub
!git clone https://github.com/doszilab/AIUPred
%cd AIUPred
!pip install torch>=2.2.0 torchvision
!pip install -r requirements.txt --no-deps || pip install numpy scipy matplotlib seaborn pandas
import sys
sys.path.append('.')
import aiupred_lib



Cloning into 'AIUPred'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 111 (delta 25), reused 30 (delta 12), pack-reused 68 (from 1)
Receiving objects: 100% (111/111), 71.60 MiB | 11.00 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Updating files: 100% (14/14), done.
/content/AIUPred/AIUPred/AIUPred
ERROR: Could not find a version that satisfies the requirement torch~=2.0.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0)
ERROR: No matching distribution found for torch~=2.0.1


In [22]:
# # Test some well-characterized IDPs
# known_idps = [
#     "MKTAYAIILQGQTQSSDILYQYLRNSAAKQASGVNQEPKPNKGTKVLDNVMAKATGKVRQGLEDLPPKSVSAENVQPGCV",  # p53 TAD
#     "MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDT",  # Calmodulin
#     # Add more known IDPs
# ]

# for idp_seq in known_idps:
#     _, mean_score = get_esm2_likelihood(idp_seq)
#     print(f"IDP: {mean_score}")

In [28]:
import random
import pandas as pd
import numpy as np
import sparrow
from sparrow.predictors.batch_predict import batch_predict
import aiupred_lib  # Added missing import

class Sequence:
    aa = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

    def __init__(self, start_seq, masked_seq, boundaries, scaling_exp, scaling_rg, mask_char='X', length=None, pH=7.):
        assert start_seq is not None or length is not None, "Either initialize a starting sequence or specify a sequence length so that a random sequence is initialized!"
        self.pH = pH
        self.boundaries = boundaries
        self.masked_seq = masked_seq
        self.start_seq = start_seq
        if start_seq is None and length is not None:
            self.sequence = random.choices(self.aa, k=length)
        else:
            self.sequence = list(masked_seq)

        self.mask_char = mask_char

        masked_count = self.sequence.count(mask_char)
        self.length = len(self.sequence) - masked_count
        self.scaling_exp = scaling_exp
        self.scaling_rg = scaling_rg
        assert not (self.scaling_exp and self.scaling_rg), "Only one scaling can be used! Both cannot be True."

    def get_sequence(self):
        return ''.join(self.sequence)

    def mutate(self, mode):
        if mode == 'single_point':
            # Find all positions that are not 'X'
            valid_positions = [i for i, char in enumerate(self.sequence) if char != self.mask_char]
            if valid_positions:
                pos = random.choice(valid_positions)
                self.sequence[pos] = random.choices(self.aa, k=1)[0]
        elif mode == 'swap_full':
            # Find all positions that are not 'X'
            valid_positions = [i for i, char in enumerate(self.sequence) if char != self.mask_char]
            if len(valid_positions) >= 2:
                a, b = random.sample(valid_positions, 2)
                self.sequence[b], self.sequence[a] = self.sequence[a], self.sequence[b]
        elif mode == 'swap_expr':
            # Find all positions that are not 'X' (excluding position 0)
            valid_positions = [i for i, char in enumerate(self.sequence[1:], 1) if char != self.mask_char]
            if len(valid_positions) >= 2:
                a, b = random.sample(valid_positions, 2)
                self.sequence[b], self.sequence[a] = self.sequence[a], self.sequence[b]

    def get_nu(self, sequence):
        if self.scaling_rg:
            val_predictions = batch_predict([sequence], return_seq2prediction=True, network="scaled_rg")
        elif self.scaling_exp:
            val_predictions = batch_predict([sequence], return_seq2prediction=True, network="scaling_exponent")
        else:
            raise ValueError("Either scaling_rg or scaling_exp must be True")
        # Extract the single value from the dict_values object
        return list(val_predictions.values())[0]

class SimulatedAnnealing(Sequence):
    def __init__(self, start_seq, masked_seq, boundaries, scaling_exp, scaling_rg,
                 mutation_mode, target_compaction, original_disorder, disorder_weight=0.3,
                 compaction_weight=0.7,
                 c=0.003, gamma=0.01,  tolerance=0.01, mask_char='X', pH=7.):
        # Initialize parent class with all required parameters
        super().__init__(start_seq, masked_seq, boundaries, scaling_exp, scaling_rg,
                        mask_char=mask_char, pH=pH)
        self.embedding_model, self.regression_model, self.device = aiupred_lib.init_models('disorder')

        # Set weights BEFORE calling fitness_function
        self.disorder_weight = disorder_weight
        self.compaction_weight = compaction_weight
        self.gamma = gamma
        # Current state - start with the original sequence
        self.sequence = list(start_seq)  # Start with original, not masked
        self.current_fitness = self.fitness_function_adaptive(start_seq, original_disorder, target_compaction)
        self.best_sequence = self.sequence.copy()
        self.best_fitness = self.current_fitness
        self.current_nu = self.get_nu(self.get_sequence())

        # Parameters
        self.mode = mutation_mode
        self.target_compaction = target_compaction
        self.original_disorder = original_disorder
        self.tolerance = tolerance
        self.c = c
        self.step_count = 0


        # Optional: track history
        self.history = []
        self.accepted_moves = 0
        self.converged = False

    def fitness_function(self, sequence, original_disorder, target_compaction):
        """
        Calculate fitness function combining compaction and disorder preservation.
        Returns a value between 0 and 1, where 1 is optimal.
        """
        # Compaction fitness (0-1 scale)
        compaction_score = self.get_nu(sequence)
        compaction_deviation = abs(compaction_score - target_compaction)
        compaction_fitness = max(0.0, 1.0 - compaction_deviation / target_compaction)

        # Disorder fitness - use Mean Squared Error instead of mutual_info_score
        try:
            current_disorder = aiupred_lib.predict_disorder(sequence, self.embedding_model,
                                                          self.regression_model, self.device)

            # Calculate MSE between disorder predictions
            mse = np.mean((np.array(original_disorder) - np.array(current_disorder))**2)

            # Option 1: Exponential decay (recommended)
            # This naturally bounds the fitness between 0 and 1, with steeper penalty for higher MSE
            disorder_fitness = np.exp(-mse * 10)  # Scale factor of 10 can be adjusted


        except Exception as e:
            print(f"Error calculating disorder: {e}")
            disorder_fitness = 0.0


        _, mean_fitness= get_esm2_likelihood(sequence)
        print(mean_fitness)

        # Combine fitnesses
        total_fitness = self.compaction_weight * compaction_fitness + self.disorder_weight * disorder_fitness
        return total_fitness

    import math


    def esm2_penalty(self,  esm2_mean_log_prob, threshold=-0.5, steepness=10):
    # Smooth penalty that kicks in below threshold
        return (1 / (1 + math.exp(steepness * (esm2_mean_log_prob - threshold))))


    def fitness_function_adaptive(self, sequence, original_disorder, target_compaction):
        """
        Adaptive version that tracks MSE statistics over time.
        """
    # Initialize MSE tracking if not exists
        if not hasattr(self, 'mse_history'):
            self.mse_history = []
            self.mse_percentile_95 = 0.25  # Initial estimate


        compaction_score = self.get_nu(sequence)

        compaction_deviation = abs(compaction_score - target_compaction)
        compaction_fitness = max(0.0, 1.0 - compaction_deviation / target_compaction)


        try:
            current_disorder = aiupred_lib.predict_disorder(sequence, self.embedding_model,
                                                      self.regression_model, self.device)


            mse = np.mean((np.array(original_disorder) - np.array(current_disorder))**2)


            self.mse_history.append(mse)


            if len(self.mse_history) % 100 == 0 and len(self.mse_history) > 10:
                self.mse_percentile_95 = np.percentile(self.mse_history, 95)


            disorder_fitness = max(0.0, 1.0 - mse / self.mse_percentile_95)

        except Exception as e:
            print(f"Error calculating disorder: {e}")
            disorder_fitness = 0.0
        # _, mean = get_esm2_likelihood(sequence)
        total_fitness = self.compaction_weight * compaction_fitness + self.disorder_weight * disorder_fitness

        return total_fitness
    def distance_from_target(self):
        """Calculate distance from target compaction"""
        current_nu = self.get_nu(''.join(self.best_sequence))
        return abs(current_nu - self.target_compaction)

    def mutate(self, mode):
        """Override to ensure mutations only happen in non-masked positions"""
        if mode == 'single_point':
            # Find all positions that are mutable (not protected by mask)
            valid_positions = []
            for i, char in enumerate(self.masked_seq):
                if char != self.mask_char:  # This position is mutable
                    valid_positions.append(i)

            if valid_positions:
                pos = random.choice(valid_positions)
                self.sequence[pos] = random.choice(self.aa)

        elif mode == 'swap_full':
            # Find all mutable positions
            valid_positions = []
            for i, char in enumerate(self.masked_seq):
                if char != self.mask_char:
                    valid_positions.append(i)

            if len(valid_positions) >= 2:
                a, b = random.sample(valid_positions, 2)
                self.sequence[b], self.sequence[a] = self.sequence[a], self.sequence[b]

        elif mode == 'swap_expr':
            # Find all mutable positions (excluding position 0)
            valid_positions = []
            for i, char in enumerate(self.masked_seq[1:], 1):
                if char != self.mask_char:
                    valid_positions.append(i)

            if len(valid_positions) >= 2:
                a, b = random.sample(valid_positions, 2)
                self.sequence[b], self.sequence[a] = self.sequence[a], self.sequence[b]

    def step(self):
        # Check if already converged
        if self.converged:
            return False

        # Store current state
        old_sequence = self.sequence.copy()
        old_fitness = self.current_fitness

        # Propose new state
        self.mutate(mode=self.mode)
        new_fitness = self.fitness_function(self.get_sequence(), self.original_disorder, self.target_compaction)

        # Calculate energy difference (fitness difference, but inverted for minimization)
        delta_E = old_fitness - new_fitness

        # Accept or reject
        accept = False
        if delta_E <= 0:  # Better solution (higher fitness)
            accept = True
        else:  # Worse solution - accept with probability
            probability = np.exp(-delta_E / self.c)
            accept = probability > random.random()

        if accept:
            # Keep the new state
            self.current_fitness = new_fitness
            self.accepted_moves += 1

            # Track best solution found
            if new_fitness > self.best_fitness:
                self.best_sequence = self.sequence.copy()
                self.best_fitness = new_fitness
        else:
            # Reject - revert to old state
            self.sequence = old_sequence
            self.current_fitness = old_fitness

        # Check for convergence - use distance from target instead of absolute fitness
        target_distance = self.distance_from_target()
        if target_distance <= self.tolerance:
            self.converged = True

        # Update step count and temperature
        self.step_count += 1
        if self.step_count % (self.length * 2) == 0:
            self.c = self.c * 0.99

        # Optional: record history (lightweight)
        self.history.append({
            'step': self.step_count,
            'fitness': self.current_fitness,
            'best_fitness': self.best_fitness,
            'target_distance': target_distance,
            'current_nu': self.get_nu(self.get_sequence()),
            'accepted': accept,
            'temperature': self.c
        })

        # Print progress occasionally
        if self.step_count % 1000 == 0:
            current_nu = self.get_nu(''.join(self.best_sequence))
            print(f"Step {self.step_count}: Best fitness={self.best_fitness:.4f}, "
                  f"Current ν={current_nu:.4f}, Target={self.target_compaction}, "
                  f"Distance={target_distance:.4f}")

        return not self.converged

    def run(self, max_steps=None):
        """Run the simulated annealing until convergence or max_steps"""
        step = 0
        while True:
            if max_steps is not None and step >= max_steps:
                print(f"Stopped after {max_steps} steps (max_steps reached)")
                break

            if not self.step():  # Returns False when converged
                print(f"Converged after {self.step_count} steps! Distance from target: {self.distance_from_target():.6f}")
                break

            step += 1

    def run_until_target(self, max_steps=10000):
        """Run specifically until target is reached or max_steps"""
        return self.run(max_steps)

    def is_converged(self):
        """Check if the algorithm has converged"""
        return self.converged

    def fitness_score(self):
        """Return current best fitness score"""
        return self.best_fitness

    def get_best_solution(self):
        """Return the best solution found so far"""
        return ''.join(self.best_sequence), self.best_fitness

    def get_current_solution(self):
        """Return the current solution"""
        return ''.join(self.sequence), self.current_fitness

    def get_acceptance_rate(self):
        """Return the acceptance rate"""
        if self.step_count == 0:
            return 0
        return self.accepted_moves / self.step_count

    def get_history_df(self):
        """Return history as DataFrame"""
        return pd.DataFrame(self.history)

    def verify_motif_preservation(self):
        """Verify that protected motifs are unchanged"""
        current_seq = self.get_sequence()
        for i, (orig_char, mask_char) in enumerate(zip(self.start_seq, self.masked_seq)):
            if mask_char == self.mask_char:  # This position should be protected
                if current_seq[i] != orig_char:
                    print(f"WARNING: Protected position {i} changed from {orig_char} to {current_seq[i]}")
                    return False
        return True




In [24]:
import torch
import esm

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
# Alternative sizes: esm2_t30_150M_UR50D, esm2_t36_3B_UR50D
batch_converter = alphabet.get_batch_converter()
model.eval()

def get_esm2_likelihood(sequence):
    # Prepare sequence
    data = [("protein", sequence)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    with torch.no_grad():
        # Get logits
        results = model(batch_tokens, repr_layers=[33])
        logits = results["logits"]

        # Calculate log probabilities
        log_probs = torch.log_softmax(logits, dim=-1)

        # Get sequence likelihood (excluding special tokens)
        seq_tokens = batch_tokens[0, 1:-1]  # Remove <cls> and <eos>
        token_log_probs = log_probs[0, 1:-1, :].gather(1, seq_tokens.unsqueeze(1))

        # Sum log probabilities (or mean for per-position average)
        total_log_prob = token_log_probs.sum().item()
        mean_log_prob = token_log_probs.mean().item()

    return total_log_prob, mean_log_prob

In [25]:



def mask_sequence_with_boundaries(sequence, boundaries, buffer_size=3, mask_char='X'):
    """
    Mask a protein sequence based on motif boundaries with optional buffer zones.

    Parameters:
    -----------
    sequence : str
        The protein sequence to mask
    boundaries : dict
        Dictionary with motif names as keys and (start, end) tuples as values
        Positions should be 0-indexed, end is exclusive (Python slicing convention)
    buffer_size : int, default=3
        Number of residues to add as buffer on each side of motif
    mask_char : str, default='X'
        Character to use for masking protected regions

    Returns:
    --------
    tuple: (masked_sequence, protected_positions)
        masked_sequence: sequence with protected regions replaced by mask_char
        protected_positions: set of all protected position indices
    """

    # Create set to track all protected positions
    protected_positions = set()

    # Process each motif boundary
    for motif_name, (start, end) in boundaries.items():
        # Add buffer zones
        buffered_start = max(0, start - buffer_size)
        buffered_end = min(len(sequence), end + buffer_size)

        # Add all positions in buffered region to protected set
        for pos in range(buffered_start, buffered_end):
            protected_positions.add(pos)

        print(f"Motif '{motif_name}': positions {start}-{end-1} "
              f"-> protected {buffered_start}-{buffered_end-1}")

    # Create masked sequence
    masked_sequence = ""
    for i, aa in enumerate(sequence):
        if i in protected_positions:
            masked_sequence += mask_char
        else:
            masked_sequence += aa

    # Print summary
    total_positions = len(sequence)
    protected_count = len(protected_positions)
    mutable_count = total_positions - protected_count


    return masked_sequence, protected_positions





# def unmask_sequence_with_boundaries(original_sequence, masked_sequence, boundaries, buffer_size=3, mask_char='X'):
#     """
#     Unmask a protein sequence by restoring original residues at protected positions.

#     Parameters:
#     -----------
#     original_sequence : str
#         The original protein sequence before masking
#     masked_sequence : str
#         The masked sequence (potentially after mutations)
#     boundaries : dict
#         Dictionary with motif names as keys and (start, end) tuples as values
#         Positions should be 0-indexed, end is exclusive (Python slicing convention)
#     buffer_size : int, default=3
#         Number of residues that were added as buffer on each side of motif
#     mask_char : str, default='X'
#         Character that was used for masking protected regions

#     Returns:
#     --------
#     tuple: (unmasked_sequence, restored_positions)
#         unmasked_sequence: sequence with original residues restored at protected positions
#         restored_positions: set of all position indices that were restored
#     """

#     # Validate input sequences have same length
#     if len(original_sequence) != len(masked_sequence):
#         raise ValueError(f"Sequence length mismatch: original={len(original_sequence)}, "
#                         f"masked={len(masked_sequence)}")

#     # Create set to track all protected positions (same logic as masking)
#     protected_positions = set()

#     # Process each motif boundary
#     for motif_name, (start, end) in boundaries.items():
#         # Add buffer zones
#         buffered_start = max(0, start - buffer_size)
#         buffered_end = min(len(original_sequence), end + buffer_size)

#         # Add all positions in buffered region to protected set
#         for pos in range(buffered_start, buffered_end):
#             protected_positions.add(pos)

#         print(f"Motif '{motif_name}': positions {start}-{end-1} "
#               f"-> restoring {buffered_start}-{buffered_end-1}")

#     # Create unmasked sequence
#     unmasked_sequence = ""
#     restored_positions = set()

#     for i, (original_aa, masked_aa) in enumerate(zip(original_sequence, masked_sequence)):
#         if i in protected_positions:
#             # Restore original amino acid at protected positions
#             unmasked_sequence += original_aa
#             if masked_aa != original_aa:  # Track positions that actually changed
#                 restored_positions.add(i)
#         else:
#             # Keep the (potentially mutated) amino acid at mutable positions
#             unmasked_sequence += masked_aa

#     # Print summary
#     total_positions = len(original_sequence)
#     protected_count = len(protected_positions)
#     restored_count = len(restored_positions)
#     mutable_count = total_positions - protected_count


#     return unmasked_sequence, restored_positions



In [29]:

mutation_mode = 'single_point'
target = 0.3#@param {type:"number"}
start_seq = "VLTKTKYTHYYGKKKNKRIGRPPGGHSNLACALKKASKRRKRRKNVFVHKKKRSSASVDNTPAGSPQGSGGEDEDDPDEGDDDSLSEGSTSEQQDELQEESEMSEKKSCSSSPTQSEISTSLPPDRQRRKRELRTFSFSDDENKPPSPKEIRIEVAERLHLD"#@param {type:"string"}
pH = 7.0
tolerance = .01#@param {type:"number"}
scaling_exp = True#@param {type:"boolean"}
scaling_rg = False#@param {type:"boolean"}
buffer_size = 2#@param {type:"number"}
esm2_likelihood_penalty= 0.01#@param {type:"number"}
boundaries = {
        "MOD_GSK3_1_1":(54,61),
        "MOD_GSK3_1":(83,90),
        "MOD_GSK3_1_2":(132,139)
}


print(f"Input sequence length: {len(start_seq)}")
# print(get_esm2_likelihood(start_seq))
masked_seq, _ = mask_sequence_with_boundaries(start_seq, boundaries, buffer_size=buffer_size)

embedding_model, regression_model, device = aiupred_lib.init_models('disorder')


original_disorder= aiupred_lib.predict_disorder(start_seq, embedding_model, regression_model, device)



sa = SimulatedAnnealing(
        start_seq=start_seq,
        masked_seq=masked_seq,
        boundaries=boundaries,
        scaling_exp=scaling_exp,
        scaling_rg=scaling_rg,
        mutation_mode=mutation_mode,
        target_compaction=target,
        original_disorder=original_disorder,
        compaction_weight=0.7,
        disorder_weight=0.3,
        c=0.003,
        gamma=esm2_likelihood_penalty,
        tolerance=tolerance,
        pH=pH
    )

print(f"Starting sequence: {sa.get_sequence()}")
print(f"Starting nu: {sa.current_nu}")

sa.run_until_target()
print(sa.get_nu(sa.get_sequence()))

print(f"Best distance achieved: {sa.distance_from_target():.6f}")
print(f"Best sequence: {sa.get_best_solution()}")

Input sequence length: 162
Motif 'MOD_GSK3_1_1': positions 54-60 -> protected 52-62
Motif 'MOD_GSK3_1': positions 83-89 -> protected 81-91
Motif 'MOD_GSK3_1_2': positions 132-138 -> protected 130-140


100%|██████████| 1/1 [00:00<00:00, 321.25it/s]


Starting sequence: VLTKTKYTHYYGKKKNKRIGRPPGGHSNLACALKKASKRRKRRKNVFVHKKKRSSASVDNTPAGSPQGSGGEDEDDPDEGDDDSLSEGSTSEQQDELQEESEMSEKKSCSSSPTQSEISTSLPPDRQRRKRELRTFSFSDDENKPPSPKEIRIEVAERLHLD
Starting nu: 0.434137225151062


100%|██████████| 1/1 [00:00<00:00, 413.52it/s]


-0.4207703471183777


...

Converged after 72 steps! Distance from target: 0.008745


100%|██████████| 1/1 [00:00<00:00, 434.19it/s]


0.30874527


100%|██████████| 1/1 [00:00<00:00, 451.19it/s]

Best distance achieved: 0.008745
Best sequence: ('VLTKTRYTHGYGKKKNKRIGRMPGKHSNLWCALKKASKRRKRRKNVFVHKKKRSSASVDNTPAGSPQGNGGEDEDDEYEEFDDSLSEGSTSEQQDELQEEEEMSEMGSCPNSYTQSGIAAFAPWDPQRRKRELRTFSFSDDRNKPPSPKEIRIHVAERLHNP', np.float32(0.97162324))
